<a href="https://colab.research.google.com/github/Canboraimamoglu/FIZ437E-HM/blob/main/HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import os 
import cv2 as cv
import matplotlib.pyplot as plt


#using image as data in my google drive in google colab. For this, images are prepared for processing

destination1 = "drive/MyDrive/car_labelled"
destination2 = "drive/MyDrive/aircraft_labelled"


airplane_vectors=[]
for f in os.listdir(destination2):
    filename=os.path.join(destination2,f)
    image = cv.imread(filename)
    numpydata = np.asarray(image)
    data = np.ravel(numpydata)
    airplane_vectors.append(data)
    image = cv.imwrite(filename,image)
airplane_vectors=np.asarray(airplane_vectors)

car_vectors=[]
for f in os.listdir(destination1):
    filename=os.path.join(destination1,f)
    image = cv.imread(filename)
    numpydata = np.asarray(image)
    data = np.ravel(numpydata)
    car_vectors.append(data)
    image = cv.imwrite(filename,image)    
car_vectors=np.asarray(car_vectors)


X=airplane_vectors+car_vectors
X=np.array(X)
Y_airplane=[ 1 for i in range(250)]
Y_cars=[0 for i in range(250)]
Y=Y_airplane+Y_cars
Y=np.array(Y)



In [2]:
#firstly created support vector machine class
class SVM():


  # initiating the hyperparameters
  def __init__(self, learning_rate, no_of_iterations, lambda_parameter):

    self.learning_rate = learning_rate
    self.no_of_iterations = no_of_iterations
    self.lambda_parameter = lambda_parameter


  
  # fitting the dataset to SVM Classifier
  def fit(self, X, Y):

    # m  --> number of Data points --> number of rows
    # n  --> number of input features --> number of columns
    self.m, self.n = X.shape

    # initiating the weight value and bias value

    self.w = np.zeros(self.n)

    self.b = 0

    self.X = X

    self.Y = Y

    # implementing Gradient Descent algorithm for Optimization

    for i in range(self.no_of_iterations):
      self.update_weights()



  # function for updating the weight and bias value
  def update_weights(self):

    # label encoding
    y_label = np.where(self.Y <= 0, -1, 1)



    # gradients ( dw, db)
    for index, x_i in enumerate(self.X):

      condition = y_label[index] * (np.dot(x_i, self.w) - self.b) >= 1

      if (condition == True):

        dw = 2 * self.lambda_parameter * self.w
        db = 0

      else:

        dw = 2 * self.lambda_parameter * self.w - np.dot(x_i, y_label[index])
        db = y_label[index]


      self.w = self.w - self.learning_rate * dw

      self.b = self.b - self.learning_rate * db



  # predict the label for a given input value
  def predict(self, X):

    output = np.dot(X, self.w) - self.b
    
    predicted_labels = np.sign(output)

    y_hat = np.where(predicted_labels <= -1, 0, 1)

    return y_hat  



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
class logistic():
  def __init__(self,X,Y):
    self.X=X
    self.Y=Y
       
  def normalize(self,X):
    '''
    function to normalize feature matrix, X
    '''
    mins = np.min(X, axis = 0)
    maxs = np.max(X, axis = 0)
    rng = maxs - mins
    norm_X = 1 - ((maxs - X)/rng)
    return norm_X
 
 
  def logistic_func(self,beta, X):
    '''
    logistic(sigmoid) function
    '''
    return 1.0/(1 + np.exp(-np.dot(X, beta.T)))
 
 
  def log_gradient(self,beta, X, Y):
    '''
    logistic gradient function
    '''
    first_calc = self.logistic_func(beta, X) - Y.reshape(X.shape[0], -1)
    final_calc = np.dot(first_calc.T, X)
    return final_calc
 
 
  def cost_func(self,beta, X, Y):
    '''
    cost function, J
    '''
    log_func_v = self.logistic_func(beta, X)
    y = np.squeeze(Y)
    step1 = y * np.log(log_func_v)
    step2 = (1 - y) * np.log(1 - log_func_v)
    final = -step1 - step2
    return np.mean(final)
 
 
  def grad_desc(self,X, Y, beta, lr=.01, converge_change=.001):
    '''
    gradient descent function
    '''
    cost = self.cost_func(beta, X, Y)
    change_cost = 1
    num_iter = 1
     
    while(change_cost > converge_change):
        old_cost = cost
        beta = beta - (lr * self.log_gradient(beta, X, Y))
        cost = self.cost_func(beta, X, Y)
        change_cost = old_cost - cost
        num_iter += 1
     
    return beta, num_iter
 
 
  def pred_values(self,beta, X):
    '''
    function to predict labels
    '''
    pred_prob = self.logistic_func(beta, X)
    pred_value = np.where(pred_prob >= .5, 1, 0)
    return np.squeeze(pred_value)

In [4]:
class NeuronLayer():
    def __init__(self, number_of_neurons, number_of_inputs_per_neuron):
        self.synaptic_weights = 2 * random.random((number_of_inputs_per_neuron, number_of_neurons)) - 1


class NeuralNetwork():
    def __init__(self, layer1, layer2):
        self.layer1 = layer1
        self.layer2 = layer2

    # The Sigmoid function, which describes an S shaped curve.
    # We pass the weighted sum of the inputs through this function to
    # normalise them between 0 and 1.
    def __sigmoid(self, x):
        return 1 / (1 + exp(-x))

    # The derivative of the Sigmoid function.
    # This is the gradient of the Sigmoid curve.
    # It indicates how confident we are about the existing weight.
    def __sigmoid_derivative(self, x):
        return x * (1 - x)

    # We train the neural network through a process of trial and error.
    # Adjusting the synaptic weights each time.
    def train(self, training_set_inputs, training_set_outputs, number_of_training_iterations):
        for iteration in range(number_of_training_iterations):
            # Pass the training set through our neural network
            output_from_layer_1, output_from_layer_2 = self.think(training_set_inputs)

            # Calculate the error for layer 2 (The difference between the desired output
            # and the predicted output).
            layer2_error = training_set_outputs - output_from_layer_2
            layer2_delta = layer2_error * self.__sigmoid_derivative(output_from_layer_2)

            # Calculate the error for layer 1 (By looking at the weights in layer 1,
            # we can determine by how much layer 1 contributed to the error in layer 2).
            layer1_error = layer2_delta.dot(self.layer2.synaptic_weights.T)
            layer1_delta = layer1_error * self.__sigmoid_derivative(output_from_layer_1)

            # Calculate how much to adjust the weights by
            layer1_adjustment = training_set_inputs.T.dot(layer1_delta)
            layer2_adjustment = output_from_layer_1.T.dot(layer2_delta)

            # Adjust the weights.
            self.layer1.synaptic_weights += layer1_adjustment
            self.layer2.synaptic_weights += layer2_adjustment

    # The neural network thinks.
    def think(self, inputs):
        output_from_layer1 = self.__sigmoid(dot(inputs, self.layer1.synaptic_weights))
        output_from_layer2 = self.__sigmoid(dot(output_from_layer1, self.layer2.synaptic_weights))
        return output_from_layer1, output_from_layer2

    # The neural network prints its weights
    def print_weights(self):
        print("    Layer 1 (4 neurons, each with 3 inputs):")
        print(self.layer1.synaptic_weights)
        print("    Layer 2 (1 neuron, with 4 inputs):")
        print(self.layer2.synaptic_weights)

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.1, random_state = 2)

In [12]:
SVM_classifier=SVM(learning_rate=0.001, no_of_iterations=1000, lambda_parameter=0.01)

In [13]:
# training the SVM classifier with training data
SVM_classifier.fit(X_train, Y_train)

In [14]:
#find accuracy
def accuracy_metric(actual, predicted):
	correct = 0
	for i in range(len(actual)):
		if actual[i] == predicted[i]:
			correct += 1
	return correct / float(len(actual)) * 100.0

In [17]:
#support vector machine train accuracy
X_train_prediction = SVM_classifier.predict(X_train)
training_data_accuracy = accuracy_metric(Y_train, X_train_prediction)
print('Accuracy score on training data = ', training_data_accuracy)

Accuracy score on training data =  83.55555555555556


In [19]:
#support vector machine test accuracy
X_test_prediction = SVM_classifier.predict(X_test)
test_data_accuracy = accuracy_metric(Y_test, X_test_prediction)
print('Accuracy score on test data = ', test_data_accuracy)

Accuracy score on test data =  60.0


In [24]:
from numpy import exp, array, random, dot
random.seed(1)

    # Create layer 1 (4 neurons, each with 3 inputs)
layer1 = NeuronLayer(4, 3)

    # Create layer 2 (a single neuron with 4 inputs)
layer2 = NeuronLayer(1, 4)

    # Combine the layers to create a neural network
neural_network = NeuralNetwork(layer1, layer2)

print("Stage 1) Random starting synaptic weights: ")
neural_network.print_weights()


training_set_inputs = X_train
training_set_outputs = Y_train

    # Train the neural network using the training set.
neural_network.train(training_set_inputs, training_set_outputs, 10000)

print("Stage 2) New synaptic weights after training: ")
neural_network.print_weights()

    # Test the neural network with a new situation.
print("Stage 3) Considering a new situation [1, 1, 0] -> ?: ")
hidden_state, output = neural_network.think(array([1, 1, 0]))
print(output)

Stage 1) Random starting synaptic weights: 
    Layer 1 (4 neurons, each with 3 inputs):
[[-0.16595599  0.44064899 -0.99977125 -0.39533485]
 [-0.70648822 -0.81532281 -0.62747958 -0.30887855]
 [-0.20646505  0.07763347 -0.16161097  0.370439  ]]
    Layer 2 (1 neuron, with 4 inputs):
[[-0.5910955 ]
 [ 0.75623487]
 [-0.94522481]
 [ 0.34093502]]


ValueError: ignored

In [23]:
    #prediction of logistic regression
import numpy as np
beta = np.matrix(np.zeros(X.shape[1]))
 
# beta values after running gradient descent
beta, num_iter = grad_desc(X_train, Y_train, beta)
 
    # estimated beta values and number of iterations
print("Estimated regression coefficients:", beta)
print("No. of iterations:", num_iter)
 
    # predicted labels
y_pred = pred_values(beta, X_train)
     
    # number of correctly predicted labels
print("Correctly predicted labels:", np.sum(y == y_pred))

NameError: ignored